**Insper**  
**Redes Sociais**

# Projeto:<br/>Best Books Ever

**Jorás Oliveira**  
**Luciano Dias**  
**Tiago Seixas**


In [ ]:
!pip install scikit-learn

In [ ]:
import sys
import re
import random
import math
import itertools

import netpixi
from ast import literal_eval
import seaborn as sns
from graph_tool import centrality, spectral, clustering
from matplotlib import pyplot as plt
from netpixi.integration.gt import *
from regression.integration.gt import *
import numpy as np
import pandas as pd
import regression as reg
from sklearn.preprocessing import minmax_scale

RANDOM_SEED = 27

In [ ]:
df = pd.read_csv("books_1.Best_Books_Ever.csv", usecols=["title", "series", "rating", "numRatings", "language", "genres"])

df['rating'] = minmax_scale(df['rating'])
df['numRatings'] = minmax_scale(df['numRatings'])
df["language"] = df["language"].map(lambda x: x if x == "English" else "non-English")
df['language'] = df['language'].astype("category")
df['genres'] = df['genres'].apply(literal_eval)

df

## 1. Construção da rede

In [ ]:
SAMPLE_SIZE = 1_500

RECORTE = 9
# 1: Livros que não estão em inglês e que fazem parte de uma série
# 2: Livros que estão em inglês e que fazem parte de uma série
# 3: Livros que fazem parte de uma série
# 4: Livros que não fazem parte de uma série e que não estão em inglês
# 5: Livros que não fazem parte de uma série e estão em inglês
# 6: Livros que não fazem parte de uma série
# 7: Livros que não estão em inglês
# 8: Livros que estão em inglês
# X: Sem recorte

filtered = df

if RECORTE in (1, 2, 3):
    filtered = filtered.loc[df["series"].notna()]
if RECORTE in (4, 5, 6):
    filtered = filtered.loc[df["series"].isna()]

if RECORTE in (1, 4, 7):
    filtered = filtered.loc[df["language"] != "English"]
if RECORTE in (2, 5, 8):
    filtered = filtered.loc[df["language"] == "English"]
    
sample = filtered.sample(SAMPLE_SIZE, replace=False, random_state=RANDOM_SEED)

g = Graph(directed=False) # não dirigido
g.add_vp('rating')
g.add_vp('clustering')
g.add_ep('weight')

sample

### 1.1. Operacionalização dos Vertices

In [ ]:
for index, row in sample.iterrows():
    vertex = g.add_vertex(index)
    vertex["rating"] = row["rating"]


In [ ]:
n = g.num_vertices()

f"{n} vertices na rede."

#### 1.1.1 Histograma de pesos dos vértices

In [ ]:
sns.histplot(sample["rating"], binrange=(0, 1));

### 1.2. Operacionalização das Arestas

In [ ]:
max_edges = n * (n - 1)

f"{max_edges} arestas possíveis."

In [ ]:
def get_weight(g1, r1, n1, g2, r2, n2):
    inter, i1, i2 = np.intersect1d(g1, g2, return_indices=True)
   
    if len(inter) == 0:
        return 0.0
    
    coef_1 = 1 - max(r1, r2) + min(r1, r2)
    coef_2 = 1 - max(n1, n2) + min(n1, n2)
    coef_3 = np.power((
        sum((
                (len(g1) - i1) + (len(g2) - i2)
            ) / (len(g1) + len(g2))
        ) / len(inter)
    ), 1 / len(inter))
   
    return coef_1 * coef_2 * coef_3


In [ ]:
combinations = itertools.combinations(sample.index, 2)
weights = []

for index, (id_1, id_2) in enumerate(combinations):
    row_1 = sample.loc[id_1]
    row_2 = sample.loc[id_2]
    weight = get_weight(row_1["genres"], row_1["rating"], row_1["numRatings"],
                        row_2["genres"], row_2["rating"], row_2["numRatings"])

    if weight > 0.5:
        edge = g.add_edge(id_1, id_2)
        edge["weight"] = weight
 
        weights.append(weight)

    if (index*2) % 1_000 == 0:
        print(f"\r{index*2} {(index*200)//max_edges}%", end="")

print(f"\r{max_edges} 100%")


In [ ]:
m = g.num_edges()

f"{m} arestas na rede. Densidade de {m/max_edges:.5f}"

#### 1.2.1 Histograma de pessos das arestas

In [ ]:
sns.histplot(weights, binrange=(0, 1));

#### 1.2.2 Histograma de degree dos vertices

In [ ]:
data = gt_data(g)
sample["degree"] = pd.Series([v.total_degree() for v in g.all_vertices()], index=sample.index)

sns.histplot(sample["degree"]);

### 1.3. Limpeza da rede

In [ ]:
clear_vertexes = sample.loc[sample["degree"] <= 1].index
sample = sample.drop(clear_vertexes)

for vertex in clear_vertexes:
    g.remove_vertex(vertex)

len(clear_vertexes)

## 2.  Regressão dos dados da rede


In [ ]:
sample["clustering"] = pd.Series(clustering.local_clustering(g), index=sample.index)

for vertex, value in sample["clustering"].items():
    g.vertex_properties["clustering"][vertex] = value

sns.histplot(sample["clustering"]);

In [ ]:
result = reg.linear(data=sample, formula='rating ~ clustering + language')

result.micro_summary()

In [ ]:
result.plot_residuals()

### 1.3. Limpeza da rede

In [ ]:
m = gt_draw.sfdp_layout(g)

gt_move(g, m)
gt_save(g, 'best-books-ever.net.gz')

In [ ]:
r = netpixi.render('best-books-ever.net.gz', infinite=True);

In [ ]:
r.edge_scale('weight', 1, 10)
r.vertex_scale('clustering', 5, 20)

In [ ]:
sample["clustering"].isna().sum()